In [1]:
from gen_graph import gen_graph, validate_graph
from typing import TypedDict
import random

In [2]:
# State
class Tally(TypedDict):
    topic: str
    human_chooses_openai_count: int
    human_chooses_anthropic_count: int
    tie_count: int
    openai_response: str
    anthropic_response: str
    openai_is_first: bool

# Conditions
def is_done(state):
    return state['topic'].lower() == 'q'

# Nodes
def human_input(state):
    topic = input("Choose a joke topic, or q to quit: ")
    if not 'human_chooses_openai_count' in state:
        state['human_chooses_openai_count'] = 0
    if not 'human_chooses_anthropic_count' in state:
        state['human_chooses_anthropic_count'] = 0
    if not 'tie_count' in state:
        state['tie_count'] = 0
    tie_count = state['tie_count']
    openai_count = state['human_chooses_openai_count']
    anthropic_count = state['human_chooses_anthropic_count']
    return { "topic": topic, 'human_chooses_openai_count': openai_count, 'human_chooses_anthropic_count': anthropic_count, 'tie_count': tie_count }

def call_openai(state):
    response = openai_llm.invoke(f"tell me a joke about {state['topic']}.  Just tell the joke.  Don't introduce it, and don't explain it.  Only the joke.")
    return { 'openai_response': response.content }

def call_anthropic(state):
    response = anthropic_llm.invoke(f"tell me a joke about {state['topic']}.  Just tell the joke.  Don't introduce it, and don't explain it.  Only the joke.")
    return { 'anthropic_response': response.content }

def randomize_for_human(state):
    return { 'openai_is_first': random.choice([True, False]) }

def human_chooses(state):
    print(f"1. {state['openai_response'] if state['openai_is_first'] else state['anthropic_response']}")
    print(f"2. {state['openai_response'] if not state['openai_is_first'] else state['anthropic_response']}")
    print("3. tied")
    choice = input("choose 1,2, or 3: ")
    if choice == "1" and state['openai_is_first']:
        return { 'human_chooses_openai_count': state['human_chooses_openai_count'] + 1 }
    if choice == "1" and not state['openai_is_first']:
        return { 'human_chooses_anthropic_count': state['human_chooses_anthropic_count'] + 1 }
    if choice == "2" and state['openai_is_first']:
        return { 'human_chooses_anthropic_count': state['human_chooses_anthropic_count'] + 1 }
    if choice == "2" and not state['openai_is_first']:
        return { 'human_chooses_openai_count': state['human_chooses_openai_count'] + 1 }
    return { 'tie_count': state['tie_count'] + 1 }

def show_tally(state):
    print(f"Open AI preferred:   {state['human_chooses_openai_count']}")
    print(f"Anthropic preferred: {state['human_chooses_anthropic_count']}")
    print(f"no preference: {state['tie_count']}")
    return state
    
graph_spec = """

START(Tally) => human_input

human_input
  is_done => show_tally
  => call_openai, call_anthropic

call_openai, call_anthropic => randomize_for_human

randomize_for_human => human_chooses

human_chooses => human_input

show_tally => END
"""
graph_code = gen_graph("tally", graph_spec)
result = validate_graph(graph_spec)
# result will have "errors" if there were errors, and "solutions" if there are solutions, and "graph" representing the graph data

In [3]:
result

{'graph': {'START': {'state': 'Tally',
   'edges': [{'condition': 'true_fn', 'destination': 'human_input'}]},
  'human_input': {'state': 'Tally',
   'edges': [{'condition': 'is_done', 'destination': 'show_tally'},
    {'condition': 'true_fn', 'destination': 'call_openai, call_anthropic'}]},
  'call_openai, call_anthropic': {'state': 'Tally',
   'edges': [{'condition': 'true_fn', 'destination': 'randomize_for_human'}]},
  'randomize_for_human': {'state': 'Tally',
   'edges': [{'condition': 'true_fn', 'destination': 'human_chooses'}]},
  'human_chooses': {'state': 'Tally',
   'edges': [{'condition': 'true_fn', 'destination': 'human_input'}]},
  'show_tally': {'state': 'Tally',
   'edges': [{'condition': 'true_fn', 'destination': 'END'}]}}}

In [4]:
from typing import TypedDict, Type, List
import inspect

def check_typeddicts(names: List[str]) -> List[bool]:
    """
    Check if TypedDict classes with the given names exist in the current namespace.
    
    Args:
        names: List of class names to check
        
    Returns:
        List[bool]: List of boolean values indicating existence of each TypedDict
    """
    # Get all variables in the current namespace
    current_namespace = globals()
    
    results = []
    
    for name in names:
        # Check if name exists and is a TypedDict
        if name in current_namespace:
            obj = current_namespace[name]
            
            # Check if it's a class
            if not isinstance(obj, type):
                results.append(False)
                continue
                
            # Check if it's a TypedDict
            try:
                is_typeddict = (issubclass(obj, dict) and 
                               hasattr(obj, '__annotations__') and 
                               getattr(obj, '_is_protocol', False) is False)
                results.append(is_typeddict)
            except TypeError:
                results.append(False)
        else:
            results.append(False)
            
    return results

In [5]:
from typing import List
import inspect

def is_function_available(func_name):
    """
    Check if a function with the given name is available and is a function object.

    Args:
        func_name: The name of the function to check.

    Returns:
        bool: True if the function exists and is a function object, False otherwise.
    """
    try:
        func = eval(func_name)
        return inspect.isfunction(func)
    except NameError:
        return False

def check_functions(names: List[str]) -> List[bool]:
    """
    Check if functions with the given names exist in the current namespace.

    Args:
        names: List of function names to check.

    Returns:
        List[bool]: List indicating the existence of each function.
    """
    results = []
    for name in names:
        is_available = is_function_available(name)
        results.append(is_available)
    return results


In [6]:
check_typeddicts(['Tally', 'Cigarette', 'human_input'])

[True, False, False]

In [7]:
check_functions(['Tally', 'Cigarette', 'human_input'])

[False, False, True]

In [8]:
result

{'graph': {'START': {'state': 'Tally',
   'edges': [{'condition': 'true_fn', 'destination': 'human_input'}]},
  'human_input': {'state': 'Tally',
   'edges': [{'condition': 'is_done', 'destination': 'show_tally'},
    {'condition': 'true_fn', 'destination': 'call_openai, call_anthropic'}]},
  'call_openai, call_anthropic': {'state': 'Tally',
   'edges': [{'condition': 'true_fn', 'destination': 'randomize_for_human'}]},
  'randomize_for_human': {'state': 'Tally',
   'edges': [{'condition': 'true_fn', 'destination': 'human_chooses'}]},
  'human_chooses': {'state': 'Tally',
   'edges': [{'condition': 'true_fn', 'destination': 'human_input'}]},
  'show_tally': {'state': 'Tally',
   'edges': [{'condition': 'true_fn', 'destination': 'END'}]}}}

In [9]:
def extract_functions(graph_data):
    """
    Extract node functions and condition functions from a graph dictionary,
    excluding special cases ('START' node and 'true_fn' condition).
    Handles multiple node names separated by commas.
    
    Args:
        graph_data (dict): Dictionary containing graph structure
        
    Returns:
        tuple: (list of node functions, list of condition functions)
    """
    node_functions = set()
    condition_functions = set()
    
    # Add all keys from the graph dict as node functions, except 'START'
    for key in graph_data['graph'].keys():
        if key != 'START':
            # Split key on commas and strip whitespace
            node_functions.update(name.strip() for name in key.split(','))
    
    # Iterate through each node in the graph
    for node in graph_data['graph'].values():
        # Process each edge in the node
        for edge in node['edges']:
            # Add destination to node functions
            # Split on commas and strip whitespace
            destinations = edge['destination'].split(',')
            node_functions.update(name.strip() for name in destinations)
            
            # Add condition to condition functions if not 'true_fn'
            if edge['condition'] != 'true_fn':
                condition_functions.add(edge['condition'])
    
    return (sorted(list(node_functions)), sorted(list(condition_functions)))

In [10]:
nodes, conditions = extract_functions(result)

In [11]:
nodes

['END',
 'call_anthropic',
 'call_openai',
 'human_chooses',
 'human_input',
 'randomize_for_human',
 'show_tally']

In [12]:
check_functions(nodes)

[False, True, True, True, True, True, True]

In [13]:
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

anthropic_llm = ChatAnthropic(model="claude-3-5-sonnet-20241022")
openai_llm = ChatOpenAI(model="gpt-4o")

In [14]:
call_openai({"topic": "cheetos"})

{'openai_response': 'Why did the Cheetos go to school? They wanted to be a little "cheddar"!'}

In [15]:
exec(graph_code)

In [16]:
print(graph_code)

# GENERATED code, creates compiled graph: tally
from langgraph.graph import START, END, StateGraph

tally = StateGraph(Tally)
tally.add_node('human_input', human_input)
tally.add_node('call_openai', call_openai)
tally.add_node('call_anthropic', call_anthropic)
tally.add_node('randomize_for_human', randomize_for_human)
tally.add_node('human_chooses', human_chooses)
tally.add_node('show_tally', show_tally)
tally.add_edge(START, 'human_input')
def after_human_input(state: Tally):
    if is_done(state):
        return 'show_tally'
    return ['call_openai', 'call_anthropic']

human_input_conditional_edges = ['call_openai', 'call_anthropic', 'show_tally']
tally.add_conditional_edges('human_input', after_human_input, human_input_conditional_edges)

tally.add_edge(['call_openai','call_anthropic'], 'randomize_for_human')
tally.add_edge('randomize_for_human', 'human_chooses')
tally.add_edge('human_chooses', 'human_input')
tally.add_edge('show_tally', END)

tally = tally.compile()


In [17]:
tally.invoke({'topic': 'cheese'})

Choose a joke topic, or q to quit:  earphones


1. Why did the earphones break up? They just couldn't find common ground—one was always plugged in, and the other was constantly tangled in its own thoughts.
2. Why did the earphones get fired from their job? They kept getting tangled up in office drama.
3. tied


choose 1,2, or 3:  3
Choose a joke topic, or q to quit:  dogs


1. Why did the dog sit in the shade?  
Because he didn't want to be a hot dog!
2. What do you call a dog magician? A labracadabrador.
3. tied


choose 1,2, or 3:  2
Choose a joke topic, or q to quit:  toes


...

1. Why do programmers always mix up Halloween and Christmas? Because Oct 31 = Dec 25.
2. Why was the computer cold? It left its Windows open!
3. tied


choose 1,2, or 3:  1
Choose a joke topic, or q to quit:  iphones


1. Why did the iPhone go to therapy? It had too many emotional baggage apps.
2. Why did the iPhone go to therapy? It couldn't find its home button.
3. tied


choose 1,2, or 3:  1
Choose a joke topic, or q to quit:  hats


GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT